# UAS MACHINE LEARNING
# Implementasi LSTM untuk memprediksi harga Bitcoin

Anggota:
1.   Salsabila Karin (140810190015)
2.   Anki Prawira H (140810190055)
3.   Prianggara Hadyan (140810190065)

Kelas: A


In [ ]:
!pip install python-binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.8 MB/s 
     |████████████████████████████████| 106 kB 20.6 MB/s 
     |████████████████████████████████| 292 kB 69.5 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 


In [ ]:
!pip install pandas-ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 8.1 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=55af48b53d318067f80331be6efd1385cebd0e274b3b509bc7dc85995ebbf0bb
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta


## Fetch Bitcoin Data

In [ ]:
# mengimport library yang akan digunakan
import math
import pandas_ta as ta
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from datetime import datetime as dt
from binance import Client
import csv

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# buat objek klien dari perpustakaan binance
client = Client(None, None)

BinanceAPIException: ignored

In [ ]:
# mendefinisikan fungsi yang mengambil data historis dari koin yang diinginkan
def getdata(symbol, period, start, end):
    """
    This function gives us the historical candle data of the desired coin
    :param symbol: coin name that you want to get data
    :param period: time period of candles
    :param start: starting date of candles data
    :param end: end date of candles data
    :return: candles data
    """
    candles = client.get_historical_klines(symbol, period, start, end)
    return candles

In [ ]:
# mendefinisikan fungsi yang membuat file csv dari data historis koin yang diinginkan
def make_csv(symbol, candles):
    """
    This function makes csv file of desired coin with defined properties
    :param symbol: coin name that you want to make csv file
    :param candles: historical data of the desired coin
    :return: historical data in csv file
    """
    csvfile = open(symbol + ".csv", "a", newline="")
    cursor = csv.writer(csvfile)
    for i in candles:
        cursor.writerow(i)
    csvfile.close()

In [ ]:
# mendefinisikan fungsi yang membuat file csv dari data historis dari beberapa koin yang diinginkan
def multiple_csv(symbols, interval, start, end):
    """
    This function makes csv file for each coin in symbols parameter with defined properties
    :param symbols: list of multiple coin names that you want to make csv file
    :param interval: time period of candles (default: client.KLINE_INTERVAL_1DAY you can change the interval)
    :param start: starting date of candles data
    :param end: end date of candles data
    :return: historical data of multiple coin in csv files
    """
    for i in symbols:
        make_csv(i, getdata(i, interval, str(start), str(end)))
        print(i, "csv file is ready.")

In [ ]:
# mendefinisikan fungsi yang mengubah stempel waktu ke tanggal
def calculate_time(timestamp):
    """
    This function turns the timestamp to the date
    :param timestamp: given timestamp
    :return: date according to given timestamp
    """
    return dt.fromtimestamp(timestamp/1000)

In [ ]:
# dapatkan data historis Bitcoin dan buat csv
multiple_csv(["BTCUSDT"], client.KLINE_INTERVAL_1DAY, "8 November 2010", "20 September 2022")

In [ ]:
# membaca data historis Bitcoin sebagai kerangka data dengan nama kolom
headers = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "QAV", "NAT", "TBBAV", "TBQAV", "Ignore"]
data = pd.read_csv("BTCUSDT.csv", names=headers)
data.head()

In [ ]:
# Ubah kolom "Open Time" dan "Close Time" menjadi Tanggal
open_date = []
for i in data["Open Time"]:
    open_date.append(calculate_time(i))
data["Open Time"] = open_date

close_date = []
for i in data["Close Time"]:
    close_date.append(calculate_time(i))
data["Close Time"] = close_date
data

## Visualize the Bitcoin Data

In [ ]:
# Visualisasikan riwayat harga penutupan
plt.figure(figsize=(16, 8))
plt.title("Bitcoin Price History")
plt.plot(data["Close Time"], data["Close"])
plt.xlabel("Time", fontsize=14,)
plt.ylabel("USDT", fontsize=14)
plt.show()

## Create Train and Test Data

In [ ]:
# Buat data baru hanya dengan kolom "Close".
close = data.filter(["Close"])
# Ubah kerangka data menjadi np array
close_array = close.values
# Lihat train data len
train_close_len = math.ceil(len(close_array) * 0.8)
train_close_len

In [ ]:
# Normalisasi data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(close_array)
scaled_data

In [ ]:
# Membuat training dataset
train_data = scaled_data[0 : train_close_len, :]
# Buat X_train dan y_train
X_train = []
y_train = []
for i in range(60, len(train_data)):
    X_train.append(train_data[i - 60 : i, 0])
    y_train.append(train_data[i, 0])
    if i <= 60:
        print(X_train)
        print(y_train)

In [ ]:
#  Buat X_train dan y_train np array
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

In [ ]:
# Membuat testing dataset
test_data = scaled_data[train_close_len - 60 : , :]
# Buat X_test and y_test
X_test = []
y_test = data.iloc[train_close_len : , :]
for i in range(60, len(test_data)):
    X_test.append(test_data[i - 60 : i, 0])

In [ ]:
# convert data testing menjadi np array dan reshape daya testing
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

## Build a LSTM Model

In [ ]:
# Membangun LSTM Model
model = Sequential()

model.add(LSTM(units=512, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], 1)))


model.add(LSTM(units=256, activation='relu', return_sequences=False))


model.add(Dense(units=1))

In [ ]:
# compile LSTM model
model.compile(optimizer="Adam", loss="mean_squared_error", metrics=['mae'])

In [ ]:
# train LSTM model
model.fit(X_train, y_train,
          epochs=3,
          batch_size=100, 
          verbose=1)

In [ ]:
# prediksi dengan LSTM model
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# RMSE (Root Mean Squared Error)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print("Skor RMSE dari model:", rmse)

In [ ]:
# plot the data
train = close[:train_close_len]
valid = close[train_close_len:]
valid["Predictions"] = predictions
#visualisasi data
plt.figure(figsize=(16, 8))
plt.title("LSTM Model")
plt.xlabel("Time", fontsize=14)
plt.ylabel("USDT", fontsize=14)
plt.plot(data["Close Time"][:train_close_len], train["Close"])
plt.plot(data["Close Time"][train_close_len:], valid[["Close", "Predictions"]])
plt.legend(["Train", "Validation", "Predictions"], loc="lower right")
plt.show()